In [1]:
#
# This is the 1st script in the spider behavior analysis pipeline for AVI inputs. It goes through a folder 
# of AVI video recording chunks, and finds the moving and static frames. 
#
# The frames containing movement are then transcoded to FMF for further processing with compress_behavior_videos
#
# Static frames are already excluded in compress_behavior_videos, which takes FMF as input. 
# Therefore raw FMF recordings do not have to be pre-processed
#

# Globals

In [11]:
#DATA_DIR = 'D:/behavior/785/'
#DATA_DIR = 'D:/data/2019-04-11/raw/'

DATA_DIR = r"C:\Users\Hsin-Yi\Documents\GitHub\web_vibration"

In [2]:
N_PARALLEL = 1

# Imports

In [3]:
import os, psutil, sys, numpy as np, scipy.io, imageio
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm

In [4]:
# Add repository root to path in directory/path-independent way. Add this to every file for convenience.
import os,sys; ROOT = os.getcwd()[:os.getcwd().rfind('spider-behavior') +  len('spider-behavior')]; sys.path.append(ROOT)

# Add motmot library
sys.path.append(os.path.join(ROOT, 'libraries'))
import motmot.FlyMovieFormat.FlyMovieFormat as fmf
import motmot.ufmf.ufmf as ufmf

In [5]:
# Set process priority to lowest
psutil.Process(os.getpid()).nice(psutil.IDLE_PRIORITY_CLASS)

# Transcode AVI to FMF

In [13]:
# Gather list of output files
filesAVI = [os.path.join(DATA_DIR, x) for x in os.listdir(DATA_DIR) if x.lower().endswith('.avi')]
len(filesAVI), filesAVI[:]

(8,
 ['C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\200_snr_threshold20_grid16.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\300_snr_threshold20_grid16.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\400_snr_threshold20_grid16.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\500_snr_threshold20_grid16.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\web_200hz-009.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\web_300hz-007.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\web_400hz-003.avi',
  'C:\\Users\\Hsin-Yi\\Documents\\GitHub\\web_vibration\\web_500hz-008.avi'])

In [14]:
def transcodeAVItoFMF(fnameAVI, overwrite=False):
    try:
        # Output file name
        fnameFMF = fnameAVI[:-4] + '.fmf'

        if not os.path.exists(fnameFMF) or overwrite:
            # Open reader
            readerAVI = imageio.get_reader(fnameAVI)
            # Open output file
            writerFMF = fmf.FlyMovieSaver(fnameFMF, version=3, format='MONO8')
            # Iterate frames
            fidx = 0
            buf = []
            for frame in tqdm(readerAVI.iter_data(), leave=False):
                # Even though movies are stored in RGB, they are 
                frame = np.mean(frame, axis=2).astype(np.uint8)
                # Get timestamp
                fnameTimestamps = fnameAVI.replace('.avi','_timestamps.mat')
                # ... TODO ...
                # scipy.io.loadmat(fnameTimestamps)['filetime']
                # Add frame to buffer
                buf.append((frame, fidx))
                fidx += 1
                # If buffer grows too large, write it to disk
                # Note: Writing in a buffered way prevents inefficient disk seeking from multiple processes
                if len(buf) > 500:
                    [writerFMF.add_frame(*b) for b in buf]
                    buf = []
            # Write remaining buffer frames
            [writerFMF.add_frame(*b) for b in buf]
            del buf
            # Close files
            readerAVI.close()
            writerFMF.close()
        # Done!
        return True
    except Exception as e:
        print(fnameAVI, e)
        return False
    imageio._proc.kill()

In [15]:
transcodeAVItoFMF([x for x in filesAVI if 'hz' in x][0], overwrite=False)

<ipython-input-14-9d404704ff14>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(readerAVI.iter_data(), leave=False):


True

In [16]:
# Transcode all AVIs
overwrite = False
results = Parallel(n_jobs=N_PARALLEL)(delayed(transcodeAVItoFMF)(fnameAVI, overwrite) for fnameAVI in tqdm(filesAVI))

<ipython-input-16-9ceac6b19eae>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  results = Parallel(n_jobs=N_PARALLEL)(delayed(transcodeAVItoFMF)(fnameAVI, overwrite) for fnameAVI in tqdm(filesAVI))


<ipython-input-14-9d404704ff14>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for frame in tqdm(readerAVI.iter_data(), leave=False):
